# Use Pretrained Weight for Initialization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# Our saved dataset
!unzip 'drive/MyDrive/Image Retrieval/dataset.zip'

Streaming output truncated to the last 5000 lines.
  inflating: dataset/scenery_river/149.jpg  
  inflating: dataset/scenery_river/265.jpg  
  inflating: dataset/scenery_river/56.jpg  
  inflating: dataset/scenery_river/23.jpg  
  inflating: dataset/scenery_river/222.jpg  
  inflating: dataset/scenery_river/133.jpg  
  inflating: dataset/scenery_river/109.jpg  
  inflating: dataset/scenery_river/41.jpg  
  inflating: dataset/scenery_river/60.jpg  
  inflating: dataset/scenery_river/116.jpg  
  inflating: dataset/scenery_river/225.jpg  
  inflating: dataset/scenery_river/236.jpg  
  inflating: dataset/scenery_river/256.jpg  
  inflating: dataset/scenery_river/32.jpg  
  inflating: dataset/scenery_river/46.jpg  
  inflating: dataset/scenery_river/79.jpg  
  inflating: dataset/scenery_river/98.jpg  
  inflating: dataset/scenery_river/101.jpg  
  inflating: dataset/scenery_river/99.jpg  
  inflating: dataset/scenery_river/47.jpg  
  inflating: dataset/scenery_river/195.jpg  
  inflating: d

In [ ]:
img_height = img_width = 160

In [ ]:
batch_size = 256

train_ds = tf.keras.utils.image_dataset_from_directory(
  directory='dataset',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 28575 files belonging to 134 classes.
Using 22860 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  directory='dataset',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 28575 files belonging to 134 classes.
Using 5715 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['animal_Alligator', 'animal_Alpaca', 'animal_Anteater', 'animal_Ants', 'animal_Bats', 'animal_Bee', 'animal_Buffalo', 'animal_Butterfly', 'animal_Camel', 'animal_Cat', 'animal_Caterpillar', 'animal_Cheetah', 'animal_Chicken', 'animal_Civet', 'animal_Cock', 'animal_Cockroaches', 'animal_Coral', 'animal_Crab', 'animal_Dog', 'animal_Dragonfly', 'animal_Duck', 'animal_Elephant', 'animal_Fish', 'animal_Flies', 'animal_Frog', 'animal_Giraffe', 'animal_Jellyfish', 'animal_Lizard', 'animal_Monkey', 'animal_Mosquito', 'animal_Owl', 'animal_Pangolins', 'animal_Rabbit', 'animal_Snail', 'animal_Snake', 'animal_Spider', 'animal_Turtle', 'animal_Whale', 'animal_bear', 'animal_bird', 'animal_cows', 'animal_donkey', 'animal_flamingo', 'animal_fox', 'animal_horse', 'animal_panda', 'animal_pig', 'animal_scorpion', 'animal_shrimp', 'animal_tiger', 'furniture_Bookcase', 'furniture_Bowl', 'furniture_Clothes', 'furniture_Piano', 'furniture_Umbrella', 'furniture_ball', 'furniture_bed', 'furniture_book', 'fu

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(256, 160, 160, 3)
(256,)


In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [ ]:
# Use weights of VGG16 trained for Imagenet dataset
base_model = tf.keras.applications.VGG16(input_shape=(img_height,img_width,3),
                                               include_top=False,
                                               weights='imagenet')

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()


In [ ]:
inputs = tf.keras.Input(shape=(img_height,img_width,3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x)  # Initial weights are from VGG16
# Add our layers
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.25)(x)  # Dropout
x = tf.keras.layers.Dense(units=512, activation='relu')(x)
x = tf.keras.layers.Dense(units=256, activation='relu')(x)
outputs = tf.keras.layers.Dense(units=134, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 160, 160, 3)       0         
                                                                 
 tf.__operators__.getitem_1   (None, 160, 160, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_1 (TFOpLambd  (None, 160, 160, 3)      0         
 a)                                                              
                                                                 
 vgg16 (Functional)          (None, 5, 5, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0   

In [ ]:
model.compile(tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])


In [ ]:
# Training
num_epochs = 50

checkpoint_path = "drive/MyDrive/Image Retrieval/model_imagerv.h5"

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_path,
            verbose=1,
            save_weights_only=True,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True)
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                            patience = 5,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)



history = model.fit(train_ds,epochs = num_epochs,validation_data = val_ds,callbacks=[model_checkpoint])

Epoch 1/50
90/90 [==============================] - ETA: 0s - loss: 4.8326 - accuracy: 0.0126
Epoch 1: val_accuracy improved from -inf to 0.01715, saving model to drive/MyDrive/Colab Note/model_imagerv.h5
90/90 [==============================] - 230s 2s/step - loss: 4.8326 - accuracy: 0.0126 - val_loss: 4.7414 - val_accuracy: 0.0171
Epoch 2/50
90/90 [==============================] - ETA: 0s - loss: 4.6899 - accuracy: 0.0213
Epoch 2: val_accuracy improved from 0.01715 to 0.02747, saving model to drive/MyDrive/Colab Note/model_imagerv.h5
90/90 [==============================] - 231s 2s/step - loss: 4.6899 - accuracy: 0.0213 - val_loss: 4.5480 - val_accuracy: 0.0275
Epoch 3/50
90/90 [==============================] - ETA: 0s - loss: 4.5226 - accuracy: 0.0309
Epoch 3: val_accuracy improved from 0.02747 to 0.03727, saving model to drive/MyDrive/Colab Note/model_imagerv.h5
90/90 [==============================] - 232s 3s/step - loss: 4.5226 - accuracy: 0.0309 - val_loss: 4.3199 - val_accura

In [ ]:
model.save('drive/MyDrive/Image Retrieval/finalmodel.h5')

In [ ]:
model.load_weights('drive/MyDrive/Image Retrieval/model_imagerv.h5')

In [ ]:
model.evaluate(train_ds)

90/90 [==============================] - 117s 1s/step - loss: 0.5218 - accuracy: 0.8355


[0.5218459963798523, 0.8355205655097961]

In [ ]:
model.evaluate(val_ds)  # This dataset is hard and has a lot of noise, the model is too overfitting
# However, prediction is not our main problem. We mainly need to the feature extract of an image
# for calculating cosine similarity in our Image Retrieval system.

23/23 [==============================] - 49s 1s/step - loss: 3.6640 - accuracy: 0.4264


[3.6640126705169678, 0.4264217019081116]

## Save Feature Extraction

In [ ]:
# Load images saved as uint8 numpy array in the 'Data Crawler' part
dataX = np.load('drive/MyDrive/Image Retrieval/images.npy')

In [ ]:
dataX = dataX.astype(np.float32)  # Cast to float32
dataX.shape

(28575, 86, 128, 3)

In [ ]:
inputs = tf.keras.Input(shape=(img_height,img_width,3))
x = data_augmentation(inputs)
x = preprocess_input(x)
outputs = base_model(x)

feature_extract_model = tf.keras.Model(inputs, outputs)

In [ ]:
feature_extract_model.compile(tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])


In [ ]:
r = 18
feature_extract_model.weights[r] - model.weights[r]  # Check if all elements are 0

<tf.Tensor: shape=(3, 3, 512, 512), dtype=float32, numpy=
array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]],


       [[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.]

In [ ]:
# Save weights of feature extraction model for safety
feature_extract_model.save_weights("drive/MyDrive/Image Retrieval/feature_extraction_weights.h5")

In [ ]:
#data_features = None

for i in range(25000, 28575):
  img = tf.image.resize(dataX[i], (img_height, img_width))
  img = tf.expand_dims(img, axis=0)

  pred = feature_extract_model.predict(img)
  if i == 25000:
    data_features_2 = pred
  else:
    data_features_2 = np.concatenate([data_features_2, pred], axis = 0)
  print(i)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 18ms/step
26075
1/1 [==============================] - 0s 17ms/step
26076
1/1 [==============================] - 0s 19ms/step
26077
1/1 [==============================] - 0s 24ms/step
26078
1/1 [==============================] - 0s 21ms/step
26079
1/1 [==============================] - 0s 18ms/step
26080
1/1 [==============================] - 0s 16ms/step
26081
1/1 [==============================] - 0s 18ms/step
26082
1/1 [==============================] - 0s 17ms/step
26083
1/1 [==============================] - 0s 16ms/step
26084
1/1 [==============================] - 0s 16ms/step
26085
1/1 [==============================] - 0s 18ms/step
26086
1/1 [==============================] - 0s 20ms/step
26087
1/1 [==============================] - 0s 20ms/step
26088
1/1 [==============================] - 0s 18ms/step
26089
1/1 [==============================] - 0s 17ms/step
26090
1/1 [================

In [ ]:
# Saved computed data features
np.save('drive/MyDrive/Image Retrieval/data_features_train.npy', data_features_2)